In [1]:
import pandas as pd
import numpy as np
import joblib
import lightgbm as lgb
from catboost import CatBoostRegressor
from tensorflow.keras.models import load_model
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import optuna



import pickle

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# モデルの読み込み（ファイル名は例）
lgb_model = joblib.load('./output/モデル/lgb_model.pkl')
nn_model = joblib.load('./output/モデル/nn_model.pkl')
cat_model = joblib.load('./output/モデル/cat_model.pkl')

In [3]:
train_df = pd.read_csv('./output/中間データ/train_preprocessed.csv')
train_df.head()

,datetime,y,week,soldout,name,kcal,remarks,payday,weather,precipitation,...,y_rolling_mean_7,y_rolling_max_7,y_rolling_min_7,y_rolling_mean_14,y_rolling_max_14,y_rolling_min_14,kcal_missing_flag,weather_group,event_キャリアアップ支援セミナー,event_ママの会
0,2013-11-18,90.0,-1.449157,0,厚切りイカフライ,-352.889193,0,0.0,-1.231166,-0.174563,...,NaN,NaN,NaN,NaN,NaN,NaN,1,-0.89977,False,False
1,2013-11-19,101.0,-0.734930,1,手作りヒレカツ,-352.889193,0,0.0,-1.231166,-0.174563,...,NaN,NaN,NaN,NaN,NaN,NaN,1,-0.89977,False,False
2,2013-11-20,118.0,-0.020702,0,白身魚唐揚げ野菜あん,-352.889193,0,0.0,-1.231166,-0.174563,...,NaN,NaN,NaN,NaN,NaN,NaN,1,-0.89977,False,False
3,2013-11-21,120.0,0.693525,1,若鶏ピリ辛焼,-352.889193,0,0.0,-1.231166,-0.174563,...,NaN,NaN,NaN,NaN,NaN,NaN,1,-0.89977,False,False
4,2013-11-22,130.0,1.407752,1,ビッグメンチカツ,-352.889193,0,0.0,-1.231166,-0.174563,...,NaN,NaN,NaN,NaN,NaN,NaN,1,-0.89977,False,False


In [4]:
X_stack = train_df.drop(columns=['y', 'name', 'datetime']).astype(np.float32)
y_stack = train_df['y']

In [5]:
# 欠損除去（必要に応じて）
mask = ~X_stack.isna().any(axis=1)
X_stack = X_stack[mask]
y_stack = y_stack[mask]

# 各モデルの予測値
lgb_pred = lgb_model.predict(X_stack)
cat_pred = cat_model.predict(X_stack)
nn_pred = nn_model.predict(X_stack).flatten()

# スタッキング入力（特徴量）
X_meta = np.vstack([lgb_pred, cat_pred, nn_pred]).T

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


In [6]:
def objective(trial):
    alpha = trial.suggest_float("alpha", 1e-4, 10.0, log=True)
    model = Ridge(alpha=alpha)
    model.fit(X_meta, y_stack)
    pred = model.predict(X_meta)
    return np.sqrt(mean_squared_error(y_stack, pred))  # RMSE

# 最適化実行
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30)

# 最良モデルの構築・評価
best_alpha = study.best_params["alpha"]
final_meta_model = Ridge(alpha=best_alpha)
final_meta_model.fit(X_meta, y_stack)
final_pred = final_meta_model.predict(X_meta)
final_rmse = np.sqrt(mean_squared_error(y_stack, final_pred))
print(f"Stacked Ridgeモデルの最終RMSE: {final_rmse:.4f}")

[I 2025-05-02 17:26:15,959] A new study created in memory with name: no-name-4aa73223-6de5-41c8-8b2a-ddc9e8d49a7d
[I 2025-05-02 17:26:15,976] Trial 0 finished with value: 0.037912118831168545 and parameters: {'alpha': 0.004000355215033098}. Best is trial 0 with value: 0.037912118831168545.
[I 2025-05-02 17:26:15,977] Trial 1 finished with value: 0.037912136627774935 and parameters: {'alpha': 0.01349177900616441}. Best is trial 0 with value: 0.037912118831168545.
[I 2025-05-02 17:26:15,978] Trial 2 finished with value: 0.03791225924046676 and parameters: {'alpha': 0.03641551902686212}. Best is trial 0 with value: 0.037912118831168545.
[I 2025-05-02 17:26:15,978] Trial 3 finished with value: 0.03792244022359613 and parameters: {'alpha': 0.31066592242031665}. Best is trial 0 with value: 0.037912118831168545.
[I 2025-05-02 17:26:15,979] Trial 4 finished with value: 0.037912348620947035 and parameters: {'alpha': 0.04647797981212668}. Best is trial 0 with value: 0.037912118831168545.
[I 2025

Stacked Ridgeモデルの最終RMSE: 0.0379


In [7]:
# モデルを保存
with open('./output/モデル/meta_model.pkl', 'wb') as f:
    pickle.dump(final_meta_model, f)